In [5]:
! pip install geodatasets

  Using cached pooch-1.8.2-py3-none-any.whl.metadata (10 kB)
Using cached pooch-1.8.2-py3-none-any.whl (64 kB)

   ---------------------------------------- 2/2 [geodatasets]



In [7]:
import pandas as pd
import geopandas as gpd
from geodatasets import get_path

df = pd.read_csv('../data/master/excel_workbooks/gt_ws_perf_for_power_bi.csv')
df.columns = df.columns.str.strip().str.lower()

cands = ['nationality','country','nation','citizenship','iso3','iso_a3']
col = next((c for c in df.columns if c in cands), None)
if col is None:
    raise ValueError("No nationality-like column found.")

# Map to ISO3 if needed (same helper as above)
def to_iso3(val):
    if pd.isna(val): return None
    s = str(val).strip()
    if len(s) == 3 and s.isalpha():
        return s.upper()
    try:
        import pycountry
        return pycountry.countries.lookup(s).alpha_3
    except Exception:
        return None

df['iso3'] = df[col].apply(to_iso3)
nat_counts = df.groupby('iso3', dropna=True).size().reset_index(name='count')

# Natural Earth countries via geodatasets (admin_0)
world = gpd.read_file(get_path("naturalearth.cultural.admin_0_countries"))
# Common 3-letter keys here are 'ADM0_A3' (sometimes 'SOV_A3' or 'ISO_A3_EH')
key = next(k for k in ['ADM0_A3','SOV_A3','ISO_A3_EH','ISO_A3'] if k in world.columns)

merged = world.merge(nat_counts, left_on=key, right_on='iso3', how='left')

ax = merged.plot(column='count',
                 figsize=(13,6),
                 legend=True,
                 cmap='Blues',
                 missing_kwds=dict(color='lightgrey', label='No data'))
ax.set_axis_off()
ax.set_title("Nationality Representation", pad=10)

ValueError: No matching item found for the query 'naturalearth.cultural.admin_0_countries'.